<a href="https://colab.research.google.com/github/mng-v/Masin-pe/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Apriori_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Kodutöö**

Kursus: Andmekaeve suurandmetest

Autorid: Dmitri Kozlov, Natalja Maksimova

Kuupäev: 19.03.2021

*Ülesanne*


> Seoste reeglite rakendus supermarket.arff andmete põhjal. Kaevandada reeglid supermarket.arff failist. Luua leitud reeglite põhjal rakendus, milles 

1) kasutaja saab lisada kaupa ostukorvi  

2) kui ostukorvi sisu vastab reegli vasakule poolele, siis annab soovituse, mis on reegli paremal poolel

3) ei anna soovitust siis, kui ostukorvis on juba soovitatud sisu.








In [2]:
import pandas as pd
from google.colab import files
df = files.upload()

Saving shopClean.csv to shopClean.csv


Me antud faili ümbersalvestasime csv formaadis. Fail sisaldab ostjate kaupa ostukorvi. Soovime koostada antud ostukorvide põhjal
soovitusreegleid APRIORI algoritmi abil, salvestada neid JSON formaadis järgmise rakendamise sammuks  (veebirakenduse loomine).

In [3]:
# загрузим данные
dataset = pd.read_csv("shopClean.csv", sep=";")
dataset = dataset.iloc[:,:-1]


# посомтрим на датасет
dataset.head()


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,10,77,140) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,department1,department2,department3,department4,department5,department6,department7,department8,department9,grocery misc,department11,baby needs,bread and cake,baking needs,coupons,juice-sat-cord-ms,tea,biscuits,canned fish-meat,canned fruit,canned vegetables,breakfast food,cigs-tobacco pkts,cigarette cartons,cleaners-polishers,coffee,sauces-gravy-pkle,confectionary,puddings-deserts,dishcloths-scour,deod-disinfectant,frozen foods,razor blades,fuels-garden aids,spices,jams-spreads,insecticides,pet foods,laundry needs,party snack foods,...,department177,department178,department179,casks white wine,casks red wine,750ml white nz,750ml red nz,750ml white imp,750ml red imp,sparkling nz,sparkling imp,brew kits/accesry,department189,port and sherry,ctrled label wine,department192,department193,department194,department195,department196,department197,department198,department199,non host support,department201,department202,department203,department204,department205,department206,department207,department208,department209,department210,department211,department212,department213,department214,department215,department216
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,t,t,NaN,t,NaN,t,NaN,NaN,t,NaN,NaN,NaN,t,t,t,t,NaN,t,NaN,t,t,NaN,NaN,NaN,NaN,NaN,NaN,t,...,NaN,NaN,NaN,NaN,NaN,t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,t,t,NaN,NaN,NaN,NaN,NaN,t,NaN,NaN,NaN,t,t,NaN,NaN,NaN,NaN,NaN,t,t,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,t,NaN,t,NaN,t,NaN,t,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN,t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,t,NaN,t,NaN,t,NaN,NaN,t,t,NaN,NaN,t,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN,NaN,NaN,t,NaN,t,NaN,t,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,t,NaN,t,t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,t,t,NaN,NaN,NaN,NaN,t,NaN,NaN,NaN,t,NaN,NaN,t,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Näeme, et on vaja muuta faili. t asemel panna tulpa nimetust, sest t- tähendab TRUE (ostajal on selline kaup).

In [4]:
datasetNew = []


for index, row in dataset.iterrows():
  newRow = []
  for i in range(len(row)):
    #for cellind, cell in row:
    
    if row[i] == "t":
      newRow.append(dataset.columns[i])
      row[i] = dataset.columns[i]
    else:
      row[i] = float("NAN")
  datasetNew.append(newRow)
print (len(datasetNew))

4627


In [5]:
df = pd.DataFrame(datasetNew) 
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,baby needs,bread and cake,baking needs,juice-sat-cord-ms,biscuits,canned vegetables,cleaners-polishers,coffee,sauces-gravy-pkle,confectionary,dishcloths-scour,frozen foods,razor blades,party snack foods,tissues-paper prd,wrapping,mens toiletries,cheese,milk-cream,margarine,small goods,fruit,vegetables,department122,750ml white nz,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,department1,canned fish-meat,canned fruit,canned vegetables,sauces-gravy-pkle,deod-disinfectant,frozen foods,pet foods,laundry needs,tissues-paper prd,deodorants-soap,haircare,milk-cream,fruit,vegetables,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,bread and cake,baking needs,juice-sat-cord-ms,biscuits,canned fruit,sauces-gravy-pkle,puddings-deserts,wrapping,health food other,small goods,dairy foods,beef,lamb,fruit,vegetables,stationary,department137,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,department1,bread and cake,baking needs,juice-sat-cord-ms,biscuits,canned vegetables,breakfast food,cleaners-polishers,frozen foods,jams-spreads,pet foods,party snack foods,tissues-paper prd,deodorants-soap,mens toiletries,cheese,margarine,dairy foods,beef,stationary,prepared meals,department137,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,bread and cake,baking needs,juice-sat-cord-ms,tea,cleaners-polishers,coffee,sauces-gravy-pkle,frozen foods,jams-spreads,laundry needs,wrapping,deodorants-soap,haircare,dental needs,meat misc,milk-cream,margarine,beef,poultry,potatoes,vegetables,condiments,small goods2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [7]:
#mõnikord iga rida täiendatakse mingi värtusega lõpuni. 
# Me realiseerisime seda iga juhuks, kui apriori ei suuda töötada tühja lahtritega. 
# ei kasutanud edasi

df2 = df
df2.fillna(method = 'ffill',axis = 1, inplace = True)

df2.head()

#df2List  = list(df2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,baby needs,bread and cake,baking needs,juice-sat-cord-ms,biscuits,canned vegetables,cleaners-polishers,coffee,sauces-gravy-pkle,confectionary,dishcloths-scour,frozen foods,razor blades,party snack foods,tissues-paper prd,wrapping,mens toiletries,cheese,milk-cream,margarine,small goods,fruit,vegetables,department122,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz,750ml white nz
1,department1,canned fish-meat,canned fruit,canned vegetables,sauces-gravy-pkle,deod-disinfectant,frozen foods,pet foods,laundry needs,tissues-paper prd,deodorants-soap,haircare,milk-cream,fruit,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables
2,bread and cake,baking needs,juice-sat-cord-ms,biscuits,canned fruit,sauces-gravy-pkle,puddings-deserts,wrapping,health food other,small goods,dairy foods,beef,lamb,fruit,vegetables,stationary,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137
3,department1,bread and cake,baking needs,juice-sat-cord-ms,biscuits,canned vegetables,breakfast food,cleaners-polishers,frozen foods,jams-spreads,pet foods,party snack foods,tissues-paper prd,deodorants-soap,mens toiletries,cheese,margarine,dairy foods,beef,stationary,prepared meals,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137,department137
4,bread and cake,baking needs,juice-sat-cord-ms,tea,cleaners-polishers,coffee,sauces-gravy-pkle,frozen foods,jams-spreads,laundry needs,wrapping,deodorants-soap,haircare,dental needs,meat misc,milk-cream,margarine,beef,poultry,potatoes,vegetables,condiments,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2,small goods2


In [8]:
#unikaalsed tooded
records = []
for i in range(0, df.shape[0]):
    records.append([str(df.values[i,j]) for j in range(0, df.shape[1])])

In [9]:

print(records[1])

['department1', 'canned fish-meat', 'canned fruit', 'canned vegetables', 'sauces-gravy-pkle', 'deod-disinfectant', 'frozen foods', 'pet foods', 'laundry needs', 'tissues-paper prd', 'deodorants-soap', 'haircare', 'milk-cream', 'fruit', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables', 'vegetables']


In [12]:
# paketi lisamine
#!pip install efficient-apriori


In [13]:
#import apriori
from efficient_apriori import apriori as aprioryEf
#%%time
# и обучимся правилам. Обратите внимание, что пороговые значения мы вибираем сами в зависимости от того,
# насколкьо "сильные" правила мы хотим получить
# min_support -- минимальный support для правил (dtype = float).
# min_confidence -- минимальное значение confidence для правил (dtype = float)
# min_lift -- минимальный lift (dtype = float)
# max_length -- максимальная длина itemset (вспоминаем про k-itemset)  (dtype = integer)

#result = list(apriori(transactions, min_support = 0.3, min_confidence = 1)) # väike reeglite arv
#result = list(apriori(transactions, min_support = 1, min_confidence = 2)) # ei olnud reegleid
itemsets, rules = aprioryEf(records, min_support=0.2, min_confidence=0.2)
print(len(itemsets))
#print(itemsets)
print(len(rules))

5
3080


In [14]:
outputRulesObj= []

Oli loodud 3080 erinevaid reegleid, millest edasi kasutame osaliselt. Valmine reegleid, millised annavad ainult üht assotsiooni.

In [15]:
#Veruson 1 
# Print out every rule with 2 items on the left hand side,
# 1 item on the right hand side, sorted by lift
rules_rhs = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 1, rules)
for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
  lhsArr = [x.strip().replace("'","").replace("(","").replace(")","") for x in str(rule.lhs).split(',')]
  rhsR  = str(rule.rhs).replace("(","").replace(")","").replace(",","").replace("'","")

  #print(rule)  # Prints the rule and its confidence, support, lift, ...
  #print(rule.confidence)
  #print(rule.support)
  #print(rule.lift)
  newObj = {
     "lhsArr": lhsArr,
     "rhsR" : rhsR,
     "confidence":rule.confidence,
     "support":rule.support,
     "lift":rule.lift
  }
  outputRulesObj.append(newObj)

In [16]:
rules_rhs3 = filter(lambda rule: len(rule.lhs) == 3 and len(rule.rhs) == 1, rules)


for rule in sorted(rules_rhs3, key=lambda rule: rule.lift):
   #print(type(rule))
   #print(rule.lhs)
   lhsArr = [x.strip().replace("'","").replace("(","").replace(")","") for x in str(rule.lhs).split(',')]
   rhsR  = str(rule.rhs).replace("(","").replace(")","").replace(",","").replace("'","")
   #print(lhsArr)
   #print(rhsR)

   newObj = {
     "lhsArr": lhsArr,
     "rhsR" : rhsR,
     "confidence":rule.confidence,
     "support":rule.support,
     "lift":rule.lift
   }

   outputRulesObj.append(newObj)
   #print(newObj.lhsArr)
   #print(newObj.rhsR)



In [17]:
rules_rhs4 = filter(lambda rule: len(rule.lhs) == 4 and len(rule.rhs) == 1, rules)


for rule in sorted(rules_rhs4, key=lambda rule: rule.lift):
   #print(type(rule))
   #print(rule.lhs)
   lhsArr = [x.strip().replace("'","").replace("(","").replace(")","") for x in str(rule.lhs).split(',')]
   rhsR  = str(rule.rhs).replace("(","").replace(")","").replace(",","").replace("'","")
   #print(lhsArr)
   #print(rhsR)

   newObj = {
     "lhsArr": lhsArr,
     "rhsR" : rhsR,
     "confidence":rule.confidence,
     "support":rule.support,
     "lift":rule.lift
   }

   outputRulesObj.append(newObj)
   #print(newObj.lhsArr)
   #print(newObj.rhsR)



In [18]:
print(len(outputRulesObj))

1095


Salvestame saadud reegleid JSON failis, mis edasi kasutame oma veebirakenduses

In [ ]:
#print(outputRulesObj[0)
import json
json_string = json.dumps(outputRulesObj, indent=4)
#print(json_string)


In [ ]:
with open('rules.json', 'w') as writefile:
    writefile.write(json_string)